# AHAB SIGN

This Jupyter Notebook describes how to sign an already existing bootable image with ML-DSA keys. This example uses a complete U-Boot binary targeting i.MX95. To build a bootable image with ELE firmware and U-Boot bootloader, see the i.MX95 AHAB with U-Boot example.
 

## 1. Prerequisites
 - SPSDK is needed with examples extension. `pip install spsdk[examples]` (Please refer to the installation documentation.)
 - This demo was tested with i.MX95 EVK board wih LPDDR5 memory and B0 revision. 

### 1.1 Requirements
 - PQC plugin for spsdk (`pip install spsdk[pqc]`).
 - A valid bootable image containing U-Boot, ELE firmware and other necessary binaries for i.MX95 (see [i.MX95 AHAB with U-BOOT](./imx95_ahab_uboot.ipynb) example, how to build it or download it from from official NXP website).
 - ML-DSA and ECC keys for signing the image (Generate with *nxpcrypto* tool - see [Keys](../../crypto/keys.ipynb) example).


In [1]:
from spsdk.utils.jupyter_utils import YamlDiffWidget

# This env variable sets colored logger output to STDOUT
# Execute this cell to enable execution of the ! line magic
%env JUPYTER_SPSDK=1
%alias execute echo %l && %l
%alias_magic ! execute

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


## 2. AHAB signing

### 2.1 AHAB sign template

We can generate a template for signing using the nxpimage ahab get-template command with `--sign` or `-s` parameter. The command will generate a template which is YAML file that contains information about signature and/or encryption configuration. 

```bash
nxpimage ahab get-template -f mimx9596 -o ahab_sign_template.yaml --sign
```

### 2.2 Signing the image

AHAB image can be signed using the config file and command:
```bash
nxpimage ahab sign -c inputs/mx95_ahab_sign.yaml -b inputs/flash.bin -o outputs/flash_signed.bin -fs outputs/fuse-script
```

As you can see, you can also specify the directory where the fuse script will be saved using the `-fs` parameter. 

In [2]:
YamlDiffWidget("inputs/mx95_ahab_sign.diffc").html

nxpimage ahab get-template -f mimx9596 -o workspace/ahab_sign_template.yaml -s --force 
Creating workspace/ahab_sign_template.yaml template file.


In [3]:
UNSIGNED_BOOTIMG = "inputs/flash.bin"
SIGNED_BOOTIMG = "outputs/flash_signed.bin"
AHAB_SIGN_CFG = "inputs/mx95_ahab_sign.yaml"
FUSE_SCRIPT_DIR = "outputs/fuse-script"

VERBOSITY = "-v"

# Sign the image
%! nxpimage $VERBOSITY ahab sign -c $AHAB_SIGN_CFG -b $UNSIGNED_BOOTIMG -o $SIGNED_BOOTIMG -fs $FUSE_SCRIPT_DIR -m serial_downloader --force

nxpimage -v ahab sign -c inputs/mx95_ahab_sign.yaml -b inputs/flash.bin -o outputs/flash_signed.bin -fs outputs/fuse-script -m serial_downloader --force 
INFO:spsdk.image.ahab.utils:Parsed Bootable image memory map: 
┌──0x0000_0000─ Bootable Image for mimx9596, Revision: latest ──┐
│                  Size: 2.7 MiB; 2,817,024 B                   │
│           Memory type: MemoryType.SERIAL_DOWNLOADER           │
│                        Pattern: zeros                         │
│┌──0x0000_0000─ primary_image_container_set ──────────────────┐│
││                 Size: 886.0 kiB; 907,264 B                  ││
││          AHAB Image for mimx9596, Revision: latest          ││
││                       Pattern: zeros                        ││
││┌──0x0000_0000─ AHAB Containers ────────────────────────────┐││
│││                 Size: 48.0 kiB; 49,152 B                  │││
│││                   AHAB Containers block                   │││
│││                      Pattern: zeros                  

## 3 Download image
### 3.1 Download AHAB image
Set the boot mode to Cortex-M Serial Downloader (1001 on EVK boot switch) and download the files using the UUU tool

In [4]:
%! nxpuuu $VERBOSITY write -f mimx9596 -b emmc $SIGNED_BOOTIMG

nxpuuu -v write -f mimx9596 -b emmc outputs/flash_signed.bin 
SDPS: boot -f outputs/flash_signed.bin


SDPV: write -f outputs/flash_signed.bin -skipspl


SDPV: jump


Success


## 4. Write fuses

We need to write SRKH (Super Root Key Hash) to fuses. This can be done using the *nxpele* tool.
When the signed AHAB image is exported, a ".bcf" script is generated.

**Note that you need to burn SRKH and also PQC SRKH fuses in this example, because we are using ML-DSA65 keys.**

The scripts can be executed using the *nxpele batch* command:

```bash
nxpele -f mimx9596 batch ./outputs/fuse-script/ahab_oem0_srk0_hash_nxpele.bcf
```

and 

```bash
nxpele -f mimx9596 batch ./outputs/fuse-script/ahab_oem0_srk1_hash_nxpele.bcf
```

## 5. Boot and test 
Now change the boot mode to Cortex-M eMMC (1010 on EVK boot switch) and reset the board. Find the serial port that belongs to U-Boot console and interrupt the boot process. 
When the console is switched to the U-Boot menu, we can use the *nxpele* tool to communicate with the ELE.


In [ ]:
%! nxpele -f mimx9596 -p COM3 -d uboot_serial get-info

nxpele -f mimx9596 -p COM3 -d uboot_serial get-info 
ELE get info ends successfully:
Command:              0xda
Version:              3
Length:               256
SoC ID:               MX95 - 0x9500
SoC version:          B000
Life Cycle:           OEM_CLSD - 0x0040
SSSM state:           4
Attest API version:   0
UUID:                 c9d88ddf5ace444da168dd2a520630e1
SHA256 ROM PATCH:     72d02b666a524aca0a3162accb4c8de7af33083fc2faefb249d87c7de1437c81
SHA256 FW:            9fdfd92f6f1a6e222362b4a9b5110a964df304a66f91e343d25c3be6a88b75c9
Advanced information:
  OEM SRKH:           41a3ab5e956ff6649e843f2686ada51840415d2fa2b6508960098d5bf6f41f27eafeff21a88f3c4b2ccf08895c1440f4b2675fb0bfe5675333b5fcbb3fba9a45
  CSAL state:         EdgeLock secure enclave random context initialization succeed - 0x02
  TRNG state:         TRNG entropy is valid and ready to be read - 0x03
  OEM PQC SRKH:       643dcc626197897115d7cce1ec079c479399af59cd46b9b45b6578fabe86d20e788683151239d8128c4f8ccc871ea86c9d8e